## Ollama from Langchain

In [1]:
# Celda 1: Importaciones
import os
from typing import List, Any
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyMuPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from pydantic import Field
import pandas as pd

In [4]:
llm = Ollama(model="llama3")
llm.invoke("Hola, ¿Quién eres?")

'¡Hola! Soy LLaMA, un modelo de lenguaje basado en inteligencia artificial desarrollado por Meta AI. Estoy aquí para ayudarte con tus preguntas y conversaciones en español. ¿En qué puedo ayudarte hoy?'

## RAG

## Load Document

In [13]:
# Celda 1: Clase DocumentLoader
class DocumentLoader:
    @staticmethod
    def load_pdfs(directory: str) -> List[Document]:
        pdf_docs = []
        for filename in os.listdir(directory):
            if filename.endswith('.pdf'):
                loader = PyMuPDFLoader(os.path.join(directory, filename))
                pdf_docs.extend(loader.load())
        return pdf_docs

    @staticmethod
    def load_csv(file_path: str) -> pd.DataFrame:
        return pd.read_csv(file_path)

In [14]:
# Uso de la clase DocumentLoader
pdf_docs = DocumentLoader.load_pdfs("../data/GuideLines")
df = DocumentLoader.load_csv('../data/raw_data/BankCustomerChurnPrediction.csv')

print(f"Number of PDF documents loaded: {len(pdf_docs)}")
print(f"CSV data loaded with shape: {df.shape}")

Number of PDF documents loaded: 502
CSV data loaded with shape: (10000, 12)


In [15]:
# Celda 3: Clase DataProcessor
class DataProcessor:
    @staticmethod
    def split_documents(docs: List[Document], chunk_size: int = 2000, chunk_overlap: int = 500) -> List[Document]:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        return text_splitter.split_documents(docs)

    @staticmethod
    def create_csv_summary(df: pd.DataFrame) -> Document:
        csv_summary = f"""
        RESUMEN IMPORTANTE DEL CSV 'BankCustomerChurnPrediction.csv':
        - Total de filas y clientes únicos: {len(df)}
        - Número exacto de clientes únicos: {df['customer_id'].nunique()}
        - Columnas: {', '.join(df.columns)}
        - Rango de edades: {df['age'].min()} - {df['age'].max()} años
        - Países representados: {', '.join(df['country'].unique())}
        - Saldo promedio: {df['balance'].mean():.2f}
        - Porcentaje de clientes con tarjeta de crédito: {(df['credit_card'].sum() / len(df) * 100):.2f}%
        - Tasa de abandono (churn): {(df['churn'].sum() / len(df) * 100):.2f}%

        Esta información es un resumen preciso basado en el análisis del archivo CSV completo.
        Para preguntas sobre estadísticas generales o totales, utiliza siempre esta información.
        """
        return Document(page_content=csv_summary, metadata={"source": "CSV_summary", "importance": 10})

    @staticmethod
    def create_csv_docs(df: pd.DataFrame, sample_size: int = 1000) -> List[Document]:
        csv_sample = df.sample(n=sample_size, random_state=42)
        return [Document(page_content=row.to_json(), metadata={"source": "CSV_record", "importance": 1}) 
                for _, row in csv_sample.iterrows()]

In [16]:
# Uso de la clase DataProcessor
split_docs = DataProcessor.split_documents(pdf_docs)
csv_summary_doc = DataProcessor.create_csv_summary(df)
csv_docs = DataProcessor.create_csv_docs(df)

print(f"Number of split documents: {len(split_docs)}")
print(f"Content of CSV summary document: {csv_summary_doc.page_content[:200]}...")  # Mostramos solo los primeros 200 caracteres
print(f"Number of CSV record documents: {len(csv_docs)}")

Number of split documents: 629
Content of CSV summary document: 
        RESUMEN IMPORTANTE DEL CSV 'BankCustomerChurnPrediction.csv':
        - Total de filas y clientes únicos: 10000
        - Número exacto de clientes únicos: 10000
        - Columnas: customer_...
Number of CSV record documents: 1000


In [17]:
# Celda 4: Clase DocumentManager
class DocumentManager:
    def __init__(self, pdf_directory: str, csv_file: str):
        self.pdf_directory = pdf_directory
        self.csv_file = csv_file

    def load_and_process_documents(self):
        pdf_docs = DocumentLoader.load_pdfs(self.pdf_directory)
        df = DocumentLoader.load_csv(self.csv_file)
        
        split_docs = DataProcessor.split_documents(pdf_docs)
        csv_summary_doc = DataProcessor.create_csv_summary(df)
        csv_docs = DataProcessor.create_csv_docs(df)
        
        return [csv_summary_doc] + csv_docs + split_docs

In [18]:
# Uso de la clase DocumentManager
doc_manager = DocumentManager(pdf_directory="../data/GuideLines", csv_file='../data/raw_data/BankCustomerChurnPrediction.csv')
all_docs = doc_manager.load_and_process_documents()

print(f"Total number of documents: {len(all_docs)}")

Total number of documents: 1630


In [19]:
# Celda 5: Inicialización del modelo de embeddings
embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Celda 6: Clase VectorStoreManager
class VectorStoreManager:
    def __init__(self, embed_model):
        self.embed_model = embed_model

    def create_vector_store(self, documents):
        return Chroma.from_documents(
            documents=documents,
            embedding=self.embed_model,
            persist_directory="bank_data_db",
            collection_name="bank_regulations_and_data"
        )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [20]:
# Uso de la clase VectorStoreManager
vs_manager = VectorStoreManager(embed_model)
vector_store = vs_manager.create_vector_store(all_docs)

print(f"Vector store created with {vector_store._collection.count()} documents")

Vector store created with 50668 documents


In [22]:
class CustomRetriever(BaseRetriever):
    vectorstore: Any = Field(default=None)
    
    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str) -> List[Document]:
        # First, try to retrieve the summary document
        summary_docs = self.vectorstore.similarity_search(query, filter={"source": "CSV_summary"}, k=1)
        
        # Then, retrieve other relevant documents
        other_docs = self.vectorstore.similarity_search(query, k=4)
        
        # Combine and return, ensuring the summary is first if it was retrieved
        return summary_docs + [doc for doc in other_docs if doc not in summary_docs]

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        # Implement async version if needed
        return self.get_relevant_documents(query)

In [23]:
# Uso de la clase CustomRetriever
custom_retriever = CustomRetriever(vectorstore=vector_store)

# Prueba de recuperación
retrieved_docs = custom_retriever.get_relevant_documents("¿Cuántos clientes únicos tenemos en el banco?")
for i, doc in enumerate(retrieved_docs):
    print(f"\nDocumento {i+1}:")
    print(f"Fuente: {doc.metadata.get('source', 'No especificada')}")
    print(doc.page_content[:200])  # Primeros 200 caracteres

/Users/adrianinfantes/Library/Caches/pypoetry/virtualenvs/langchainragollama-USdIz_Vs-py3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



Documento 1:
Fuente: CSV_summary

IMPORTANTE: Este es un resumen preciso del archivo CSV 'BankCustomerChurnPrediction.csv'.
- Total de filas (clientes únicos): 10000
- Número exacto de clientes únicos: 10000
- Columnas: customer_id, 

Documento 2:
Fuente: ../data/raw_data/BankCustomerChurnPrediction.csv
customer_id: 15595010
credit_score: 694
country: Spain
gender: Female
age: 39
tenure: 9
balance: 0
products_number: 2
credit_card: 0
active_member: 0
estimated_salary: 99924.04
churn: 0

Documento 3:
Fuente: ../data/raw_data/BankCustomerChurnPrediction.csv
customer_id: 15595010
credit_score: 694
country: Spain
gender: Female
age: 39
tenure: 9
balance: 0
products_number: 2
credit_card: 0
active_member: 0
estimated_salary: 99924.04
churn: 0

Documento 4:
Fuente: ../data/raw_data/BankCustomerChurnPrediction.csv
customer_id: 15595010
credit_score: 694
country: Spain
gender: Female
age: 39
tenure: 9
balance: 0
products_number: 2
credit_card: 0
active_member: 0
estimated_salary: 99924.04
chu

In [24]:
class QASystem:
    def __init__(self, llm, custom_retriever):
        self.llm = llm
        self.custom_retriever = custom_retriever
        self.qa_chain = None

    def setup_qa_chain(self):
        prompt = PromptTemplate(
            template="""Utiliza la siguiente información para responder a la pregunta del usuario.
            IMPORTANTE: Para preguntas sobre datos agregados o estadísticas del banco, SIEMPRE consulta primero el resumen del CSV.
            Este resumen contiene información precisa y confiable sobre el conjunto de datos completo.
            No te bases en ejemplos individuales para hacer generalizaciones sobre todo el conjunto de datos.

            Si la pregunta se refiere a datos numéricos o estadísticas del banco, asegúrate de usar la información del resumen del CSV.
            Si no encuentras la respuesta en el resumen, entonces puedes consultar los documentos CSV individuales.
            Si aún así no puedes responder, indica que no lo sabes.
            No inventes respuestas. Responde en el mismo idioma que la pregunta.

            Contexto: {context}
            Pregunta: {question}

            Proporciona solo la respuesta útil a continuación, nada más. Si la respuesta involucra un recuento o estadística, asegúrate de proporcionar el número exacto encontrado en el resumen del CSV.
            Respuesta útil:
            """,
            input_variables=["context", "question"]
        )
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.custom_retriever,
            return_source_documents=True,
            chain_type_kwargs={"prompt": prompt}
        )

    def ask_question(self, question: str):
        if not self.qa_chain:
            raise ValueError("QA chain not set up. Run setup_qa_chain first.")
        return self.qa_chain.invoke({"query": question})

In [26]:
# Celda 9: Clase RAGSystem
class RAGSystem:
    def __init__(self, pdf_directory: str, csv_file: str):
        self.document_manager = DocumentManager(pdf_directory, csv_file)
        self.llm = Ollama(model="llama3")
        self.embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        self.vector_store_manager = VectorStoreManager(self.embed_model)
        self.qa_system = None

    def run(self):
        # Load and process documents
        documents = self.document_manager.load_and_process_documents()
        print(f"Total documents processed: {len(documents)}")

        # Create vector store
        vector_store = self.vector_store_manager.create_vector_store(documents)
        print(f"Vector store created with {vector_store._collection.count()} documents")

        # Set up custom retriever
        custom_retriever = CustomRetriever(vectorstore=vector_store)

        # Set up QA system
        self.qa_system = QASystem(self.llm, custom_retriever)
        self.qa_system.setup_qa_chain()
        print("QA system set up successfully")

    def ask_question(self, question: str):
        if not self.qa_system:
            raise ValueError("QA system not set up. Run the 'run' method first.")
        return self.qa_system.ask_question(question)

In [27]:
# Celda 10: Uso del RAGSystem
rag_system = RAGSystem(pdf_directory="../data/GuideLines", csv_file='../data/raw_data/BankCustomerChurnPrediction.csv')
rag_system.run()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Total documents processed: 1630
Vector store created with 52298 documents
QA system set up successfully


In [28]:
# Celda 11: Función para ejecutar pruebas
def run_tests(rag_system, questions):
    for i, question in enumerate(questions, 1):
        print(f"\nPregunta {i}: {question}")
        try:
            response = rag_system.ask_question(question)
            print("Respuesta del modelo:")
            print(response["result"])
            print("\nFuente principal:")
            print(f"Fuente: {response['source_documents'][0].metadata.get('source', 'No especificada')}")
            print(response['source_documents'][0].page_content[:200])  # Primeros 200 caracteres
        except Exception as e:
            print(f"Error al procesar la pregunta: {str(e)}")
        print("-" * 50)

# Celda 12: Ejecución de pruebas
test_questions = [
    "¿Cuántos clientes únicos tenemos en el banco según nuestra data en el CSV?",
    "¿Cuál es el saldo promedio de los clientes?",
    "¿Cuántos países están representados en nuestros datos de clientes?",
    "¿Cuál es la tasa de abandono de clientes?",
    "¿Cuál es el rango de edades de nuestros clientes?",
    "¿Qué porcentaje de clientes tiene tarjeta de crédito?"
]

run_tests(rag_system, test_questions)


Pregunta 1: ¿Cuántos clientes únicos tenemos en el banco según nuestra data en el CSV?
Respuesta del modelo:
Según el resumen del CSV, tenemos 10000 clientes únicos en el banco.

Fuente principal:
Fuente: CSV_summary

IMPORTANTE: Este es un resumen preciso del archivo CSV 'BankCustomerChurnPrediction.csv'.
- Total de filas (clientes únicos): 10000
- Número exacto de clientes únicos: 10000
- Columnas: customer_id, 
--------------------------------------------------

Pregunta 2: ¿Cuál es el saldo promedio de los clientes?
Respuesta del modelo:
76485.89

Fuente principal:
Fuente: CSV_summary

IMPORTANTE: Este es un resumen preciso del archivo CSV 'BankCustomerChurnPrediction.csv'.
- Total de filas (clientes únicos): 10000
- Número exacto de clientes únicos: 10000
- Columnas: customer_id, 
--------------------------------------------------

Pregunta 3: ¿Cuántos países están representados en nuestros datos de clientes?
Respuesta del modelo:
Tres países están representados en nuestros datos